In [ ]:
import time
import concurrent.futures
import papermill as pm
import psutil
import os
from datetime import datetime, timedelta
from tqdm import tqdm

notebooks = ['Bravo_Selenium.ipynb', 'Neptun_Scrape.ipynb', 'BazarStore_new.ipynb']

interval_in_seconds = 86400  # 24 hours
last_execution_time = datetime.now()

def execute_notebook(notebook):
    try:
        pm.execute_notebook(notebook, notebook)
    except Exception as e:
        print(f"Error running notebook {notebook}: {e}")

if __name__ == "__main__":
    # Execute notebooks on first run
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for notebook in notebooks:
            futures.append(executor.submit(execute_notebook, notebook))
        
        with tqdm(total=len(futures), desc="Executing notebooks") as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(1)

    print("Papermill execution complete.")

    while True:
        # Check elapsed time since last execution
        elapsed_time = (datetime.now() - last_execution_time).total_seconds()
        if elapsed_time < interval_in_seconds:
            time.sleep(interval_in_seconds - elapsed_time)

        # Execute notebooks
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = []
            for notebook in notebooks:
                futures.append(executor.submit(execute_notebook, notebook))
            
            with tqdm(total=len(futures), desc="Executing notebooks") as pbar:
                for _ in concurrent.futures.as_completed(futures):
                    pbar.update(1)

        print("Papermill execution complete.")

        # Monitor resource usage
        with tqdm(desc="Monitoring resources") as pbar:
            while True:
                process = psutil.Process(os.getpid())
                cpu_percent = process.cpu_percent()
                mem_usage = process.memory_info().rss
                pbar.set_postfix({"CPU": f"{cpu_percent:.1f}%", "Memory": f"{mem_usage / 1024 / 1024:.1f} MB"})
                pbar.update(1)
                time.sleep(1)

                if pbar.n >= interval_in_seconds:
                    pbar.close()
                    break

        # Update last execution time
        last_execution_time = datetime.now()

        print(f"Next execution at {last_execution_time + timedelta(seconds=interval_in_seconds)}.")


Executing notebooks:   0%|          | 0/3 [00:00<?, ?it/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/17 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing notebooks: 100%|██████████| 3/3 [39:52<00:00, 797.51s/it]

Error running notebook BazarStore_new.ipynb: 
---------------------------------------------------------------------------
Exception encountered at "In [3]":
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [3], in <cell line: 15>()
     13 # Convert them back to float
     14 df1["regular_price"] = df1["regular_price"].astype(float)
---> 15 df1["sale_price"] = df1["sale_price"].astype(float)
     17 # Create a new column "endirim_faiz" which is (regular_price- sale_price)/regular_price
     18 df1["endirim_faiz"] = (df1["regular_price"] - df1["sale_price"]) / df1["regular_price"]

File ~/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:5912, in NDFrame.astype(self, dtype, copy, errors)
   5905     results = [
   5906         self.iloc[:, i].astype(dtype, copy=copy)
   5907         for i in range(len(self.columns))
   5908     ]
   5910 else:
   5911     # else